### 1. Construcción de la muestra **M**

**Objetivo:** generar una muestra representativa **M** de la población **P** (datos IEEE-CIS Fraud Detection), preservando la distribución natural de las variables de caracterización:

* **`isFraud`**  (0 = legítimo, 1 = fraude)  
* **`ProductCD`**  (W, C, R, H, S)

Creamos 10 estratos \(Mi = \{\,\text{isFraud}=x,\; \text{ProductCD}=y\,\}\).  
Para evitar sesgos:

1. **Muestreo proporcional (5 %)** dentro de cada estrato → mantiene la distribución global.  
2. **Umbral mínimo** de **150 filas** por estrato para que los grupos pequeños queden bien representados.  
3. Si un estrato tiene < 150 registros, se selecciona **todo** el estrato (no altera la proporción global de forma significativa dada la población de 590 k).

El resultado es un DataFrame **`M`** y 10 particiones **`Mi`** cacheados para uso posterior.


In [1]:
# ------------- Sección 1 • Código: Construcción de la muestra M -------------

from pyspark.sql import SparkSession, functions as F
from pathlib import Path

# 1️⃣  Spark session
spark = (
    SparkSession.builder
    .appName("Actividad4_MuestraM")
    .config("spark.driver.memory", "12g")
    .getOrCreate()
)

# 2️⃣  Rutas de los CSV
DATA_DIR = Path("/Users/rocha/Desktop/An-lisis-de-Grandes-Vol-menes-de-Datos")
path_tx  = DATA_DIR / "train_transaction.csv"
path_id  = DATA_DIR / "train_identity.csv"

# 3️⃣  Carga y unión
tx_df = spark.read.csv(str(path_tx), header=True, inferSchema=True)
id_df = spark.read.csv(str(path_id), header=True, inferSchema=True)

full_df = tx_df.join(id_df, on="TransactionID", how="left").cache()
print(f"Transacciones totales en población P: {full_df.count():,}")

# 4️⃣  Generar clave de estrato  (isFraud_ProductCD)
full_df = full_df.withColumn(
    "stratum", F.concat_ws("_", F.col("isFraud").cast("string"), F.col("ProductCD"))
)

# 5️⃣  Conteo por estrato
stratum_counts = (
    full_df.groupBy("stratum").count()
           .orderBy("stratum")
           .collect()
)

# 6️⃣  Calcular fracciones: 5 % o lo necesario para ≥150 filas
SAMPLE_BASE = 0.05            # 5 %
MIN_PER_STRATUM = 150

fractions = {}
for row in stratum_counts:
    s = row["stratum"]
    n = row["count"]
    frac = max(SAMPLE_BASE, MIN_PER_STRATUM / n)
    fractions[s] = min(frac, 1.0)   # nunca >1

print("\nFracciones aplicadas por estrato:")
for k, v in fractions.items():
    print(f"  {k:6s} → {v:.3f}")

# 7️⃣  Muestreo estratificado
M_df = (
    full_df.sampleBy("stratum", fractions=fractions, seed=42)
           .cache()
           .drop("stratum")
)

print(f"\nTamaño final de la muestra M: {M_df.count():,}")

# 8️⃣  Crear y almacenar cada partición Mi en un diccionario
partitions = {}
for row in stratum_counts:
    key = row["stratum"]
    label, prod = key.split("_")
    part_df = M_df.filter((F.col("isFraud") == int(label)) & (F.col("ProductCD") == prod)).cache()
    partitions[key] = part_df
    print(f"   Mi={key:6s}  →  {part_df.count():4d} filas")

# 9️⃣  Chequeo rápido: proporción de fraude en M
print("\nDistribución isFraud en M:")
M_df.groupBy("isFraud").count().show()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/08 21:17:25 WARN Utils: Your hostname, Franciscos-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.100.73 instead (on interface en0)
25/06/08 21:17:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/08 21:17:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/08 21:17:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Transacciones totales en población P: 590,540

Fracciones aplicadas por estrato:
  0_C    → 0.050
  0_H    → 0.050
  0_R    → 0.050
  0_S    → 0.050
  0_W    → 0.050
  1_C    → 0.050
  1_H    → 0.095
  1_R    → 0.105
  1_S    → 0.219
  1_W    → 0.050



Tamaño final de la muestra M: 29,966
   Mi=0_C     →  3082 filas
   Mi=0_H     →  1610 filas
   Mi=0_R     →  1785 filas
   Mi=0_S     →   554 filas
   Mi=0_W     →  21604 filas
   Mi=1_C     →   409 filas
   Mi=1_H     →   139 filas
   Mi=1_R     →   184 filas
   Mi=1_S     →   149 filas
   Mi=1_W     →   450 filas

Distribución isFraud en M:
+-------+-----+
|isFraud|count|
+-------+-----+
|      1| 1331|
|      0|28635|
+-------+-----+



## 2. Construcción Train – Test  

Dividimos cada partición \(M_i\) (definida por `isFraud` & `ProductCD`) en:

| Conjunto | Propósito | Porcentaje |
|----------|-----------|------------|
| **Tᵣᵢ**   | Entrenamiento | 80 % |
| **Tˢᵢ**   | Prueba        | 20 % |

Pasos:

1. **Muestreo estratificado** dentro de cada \(M_i\) → preserva la tasa real de fraude.  
2. Verificamos que `Tᵣᵢ ∩ Tˢᵢ = ∅` y `Tᵣᵢ ∪ Tˢᵢ = M_i`.  
3. Unimos todos los `Tᵣᵢ` ⇒ `train_df` y todos los `Tˢᵢ` ⇒ `test_df`, ambos cacheados.  
4. Mostramos conteos globales y distribución final de fraude.


In [2]:
# ------------- Sección 2 • Código: Train/Test por partición -------------

from pyspark.sql import functions as F

TRAIN_FRAC = 0.8
fractions  = {0: TRAIN_FRAC, 1: TRAIN_FRAC}

train_parts, test_parts = [], []

for key, part_df in partitions.items():
    # 1️⃣ Muestreo estratificado
    tri = part_df.sampleBy("isFraud", fractions, seed=42).cache()
    tsi = part_df.subtract(tri).cache()
    # 2️⃣ Verificación
    assert tri.count() + tsi.count() == part_df.count()
    assert tri.intersect(tsi).count() == 0
    train_parts.append(tri)
    test_parts.append(tsi)
    print(f"{key:6s} → Train: {tri.count():4d} | Test: {tsi.count():4d}")

# 3️⃣ Unión global
train_df = train_parts[0]
for df in train_parts[1:]:
    train_df = train_df.unionByName(df)
train_df = train_df.cache()

test_df = test_parts[0]
for df in test_parts[1:]:
    test_df = test_df.unionByName(df)
test_df = test_df.cache()

# 4️⃣ Chequeo final
print(f"\nTotal Train: {train_df.count():,} | Total Test: {test_df.count():,}")
print("\nDistribución isFraud en Train:")
train_df.groupBy("isFraud").count().show()
print("Distribución isFraud en Test:")
test_df.groupBy("isFraud").count().show()

# Guardamos una copia completa (se usará en clustering)
M_df = train_df.unionByName(test_df).cache()


0_C    → Train: 2459 | Test:  623


0_H    → Train: 1276 | Test:  334


0_R    → Train: 1410 | Test:  375


0_S    → Train:  449 | Test:  105


25/06/08 21:18:11 WARN DAGScheduler: Broadcasting large task binary with size 1128.1 KiB
25/06/08 21:18:12 WARN DAGScheduler: Broadcasting large task binary with size 1137.2 KiB
25/06/08 21:18:13 WARN DAGScheduler: Broadcasting large task binary with size 1247.3 KiB
25/06/08 21:18:16 WARN DAGScheduler: Broadcasting large task binary with size 1137.2 KiB


...

25/06/08 21:18:49 WARN DAGScheduler: Broadcasting large task binary with size 5.6 MiB
25/06/08 21:19:07 WARN DAGScheduler: Broadcasting large task binary with size 5.6 MiB



Total Train: 23,886 | Total Test: 6,080

Distribución isFraud en Train:


+-------+-----+
|isFraud|count|
+-------+-----+
|      0|22815|
|      1| 1071|
+-------+-----+

Distribución isFraud en Test:


25/06/08 21:19:26 WARN DAGScheduler: Broadcasting large task binary with size 5.6 MiB


+-------+-----+
|isFraud|count|
+-------+-----+
|      0| 5820|
|      1|  260|
+-------+-----+



### 3. Selección de métricas para medir calidad de resultados

Al trabajar con grandes volúmenes de datos y modelos de clasificación y clustering, elegimos:

| Tipo | Métrica                  | Descripción y justificación |
|------|--------------------------|------------------------------|
| **Supervisado** | **AUC-ROC** (Area Under ROC Curve) | Mide capacidad de distinguir clases positivas/negativas en todo rango de umbrales. Escalable y robusta en datasets desbalanceados. |
|                | **Area Under PR Curve** (PR-AUC)  | Se enfoca en precisión/recall para la clase minoritaria (fraude). Fundamental cuando la clase positiva es rara. |
|                | **F1-Score**                      | Equilibrio entre precisión y recall; útil para puntos de decisión específicos. |
|                | **Matriz de confusión**           | Proporciona conteos de TP, FP, FN, TN; base para métricas operativas. |
| **No supervisado** | **Silhouette Score**             | Mide cohesión vs separación de clústeres; eficiente de calcular en Big Data. |
|                | **Within Set Sum of Squared Errors** (WSSSE) | Evalúa compacidad interna de grupos; permite comparar distintos k. |
|                | **Tasa de fraude por clúster**    | Medida externa: % de fraudes en cada clúster; valida relevancia de agrupamientos. |

> **Nota**:  
> - Usamos **PR-AUC** junto con AUC-ROC para asegurar que optimizamos verdaderamente la detección de fraudes.  
> - Para clustering, complementamos Silhouette (interna) con la tasa de fraude por clúster (externa) para evaluar utilidad de los grupos.  
> - Todas son computables eficientemente en PySpark y paralelizables sobre la muestra de ~30 k filas.


## 4. Entrenamiento de Modelos de Aprendizaje  

**Supervisado** – *Gradient-Boosted Trees*  
* `StringIndexer` + `VectorAssembler` (`handleInvalid="keep"`).  
* Grid ligero (`maxDepth` 5 vs 8) validado con `TrainValidationSplit`.  
* Métricas: **AUC-ROC** & **PR-AUC**.

**No supervisado** – *Gaussian Mixture*  
* Variables numéricas escaladas (`StandardScaler`).  
* `k = 3` componentes.  
* Métricas: **Silhouette** & % fraude por componente.

Todos los `NaN` numéricos se rellenan con −999 👉 cero errores en Spark.  
`shuffle.partitions = 200` y `parallelism = 4` usan al máximo la CPU del M4 Pro.


In [3]:
# ------------------ 4. Entrenamiento optimizado ------------------

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.clustering import GaussianMixture
from pyspark.ml.evaluation import BinaryClassificationEvaluator, ClusteringEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

# ≥ 200 tareas shuffle para paralelismo
spark.conf.set("spark.sql.shuffle.partitions", "200")

# 🔄 Asegurar que M_df existe (unión completa de la muestra)
M_df = M_df if "M_df" in globals() else train_df.unionByName(test_df).cache()

# 🔍  Relleno definitivo anti-NaN en numéricas
def fill_numerics(df):
    num_cols = [c for c, t in df.dtypes if t in ("double","float")]
    return df.fillna(-999, subset=num_cols)

train_df  = fill_numerics(train_df)
test_df   = fill_numerics(test_df)
M_df      = fill_numerics(M_df)

# 1️⃣ Columnas
numeric_cols = [c for c, t in train_df.dtypes
                if t in ("double","float","int","bigint","long") and c != "isFraud"]
categorical_cols = [c for c in train_df.columns
                    if c not in numeric_cols + ["isFraud","TransactionID"]]

MAX_CAT = 100  # umbral de cardinalidad
small_cats = [c for c in categorical_cols
              if train_df.select(c).distinct().count() <= MAX_CAT]
large_cats = list(set(categorical_cols) - set(small_cats))
print(f"Indexaremos {len(small_cats)} columnas categóricas; "
      f"omitimos {len(large_cats)} muy grandes.")

# 2️⃣ Indexar categóricas
indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx",
                          handleInvalid="keep")
            for c in small_cats]

assembler = VectorAssembler(
    inputCols=numeric_cols + [f"{c}_idx" for c in small_cats],
    outputCol="features",
    handleInvalid="keep")

# 4️⃣ GBTClassifier
gbt = GBTClassifier(labelCol="isFraud",
                    featuresCol="features",
                    maxIter=25,
                    seed=42)

pipeline_gbt = Pipeline(stages=indexers + [assembler, gbt])

paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [4, 6])
             .addGrid(gbt.maxBins,  [128])      # seguro ≤ MAX_CAT
             .build())

tvs = TrainValidationSplit(estimator=pipeline_gbt,
                           evaluator=BinaryClassificationEvaluator(
                               labelCol="isFraud", metricName="areaUnderROC"),
                           estimatorParamMaps=paramGrid,
                           trainRatio=0.8,
                           parallelism=4,
                           seed=42)

best_model = tvs.fit(train_df).bestModel
pred_gbt   = best_model.transform(test_df)

evaluator_roc = BinaryClassificationEvaluator(labelCol="isFraud",
                                              metricName="areaUnderROC")
evaluator_pr  = BinaryClassificationEvaluator(labelCol="isFraud",
                                              metricName="areaUnderPR")

print(f"\n✅ GBT – AUC-ROC: {evaluator_roc.evaluate(pred_gbt):.4f} | "
      f"PR-AUC: {evaluator_pr.evaluate(pred_gbt):.4f}")

# 5️⃣ Gaussian Mixture
assembler_num = VectorAssembler(inputCols=numeric_cols,
                                outputCol="num_features")
scaler = StandardScaler(inputCol="num_features",
                        outputCol="scaled_features",
                        withMean=True, withStd=True)
gmm = GaussianMixture(featuresCol="scaled_features", k=3, seed=42)

pipeline_gmm = Pipeline(stages=[assembler_num, scaler, gmm])
gmm_model    = pipeline_gmm.fit(M_df)

gmm_result = gmm_model.transform(M_df).cache()
silhouette  = ClusteringEvaluator(featuresCol="scaled_features",
                                  predictionCol="prediction",
                                  metricName="silhouette").evaluate(gmm_result)
print(f"✅ GMM – Silhouette: {silhouette:.4f}")

print("\nFraude por componente GMM:")
gmm_result.groupBy("prediction", "isFraud").count()\
          .orderBy("prediction", "isFraud").show()


Indexaremos 30 columnas categóricas; omitimos 1 muy grandes.


25/06/08 21:20:49 WARN DAGScheduler: Broadcasting large task binary with size 1102.1 KiB
25/06/08 21:20:49 WARN DAGScheduler: Broadcasting large task binary with size 1102.1 KiB
25/06/08 21:21:00 WARN DAGScheduler: Broadcasting large task binary with size 1151.0 KiB
25/06/08 21:21:02 WARN DAGScheduler: Broadcasting large task binary with size 1151.0 KiB
25/06/08 21:21:06 WARN DAGScheduler: Broadcasting large task binary with size 1151.0 KiB
25/06/08 21:21:06 WARN DAGScheduler: Broadcasting large task binary with size 1151.0 KiB
25/06/08 21:21:11 WARN DAGScheduler: Broadcasting large task binary with size 1151.0 KiB
25/06/08 21:21:11 WARN DAGScheduler: Broadcasting large task binary with size 1151.0 KiB
25/06/08 21:21:15 WARN DAGScheduler: Broadcasting large task binary with size 1151.0 KiB
25/06/08 21:21:15 WARN DAGScheduler: Broadcasting large task binary with size 1151.0 KiB
25/06/08 21:21:20 WARN DAGScheduler: Broadcasting large task binary with size 1151.0 KiB
25/06/08 21:21:20 WAR


✅ GBT – AUC-ROC: 0.8755 | PR-AUC: 0.5256


25/06/08 22:10:00 WARN DAGScheduler: Broadcasting large task binary with size 6.3 MiB
25/06/08 22:10:42 WARN DAGScheduler: Broadcasting large task binary with size 6.4 MiB
25/06/08 22:11:38 WARN DAGScheduler: Broadcasting large task binary with size 6.4 MiB
25/06/08 22:12:33 WARN DAGScheduler: Broadcasting large task binary with size 6.4 MiB
25/06/08 22:13:13 WARN DAGScheduler: Broadcasting large task binary with size 6.4 MiB
25/06/08 22:13:13 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
25/06/08 22:14:02 WARN DAGScheduler: Broadcasting large task binary with size 6.4 MiB
25/06/08 22:15:24 WARN DAGScheduler: Broadcasting large task binary with size 6.4 MiB
25/06/08 22:15:24 ERROR Executor: Exception in task 3.0 in stage 3391.0 (TID 1973510)
breeze.linalg.MatrixNotSymmetricException: Matrix is not symmetric
	at breeze.linalg.package$.$anonfun$requireSymmetricMatrix$2(package.scala:152)
	at scala.collection.immutable.Range.foreach$mVc$sp(Ra

Py4JJavaError: An error occurred while calling o12426.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 3391.0 failed 1 times, most recent failure: Lost task 9.0 in stage 3391.0 (TID 1973516) (192.168.100.73 executor driver): breeze.linalg.MatrixNotSymmetricException: Matrix is not symmetric
	at breeze.linalg.package$.$anonfun$requireSymmetricMatrix$2(package.scala:152)
	at scala.collection.immutable.Range.foreach$mVc$sp(Range.scala:192)
	at breeze.linalg.package$.$anonfun$requireSymmetricMatrix$1(package.scala:150)
	at scala.collection.immutable.Range.foreach$mVc$sp(Range.scala:192)
	at breeze.linalg.package$.requireSymmetricMatrix(package.scala:150)
	at breeze.linalg.eigSym$.breeze$linalg$eigSym$$doEigSym(eig.scala:140)
	at breeze.linalg.eigSym$EigSym_DM_Impl$.apply(eig.scala:113)
	at breeze.linalg.eigSym$EigSym_DM_Impl$.apply(eig.scala:111)
	at breeze.generic.UFunc.apply(UFunc.scala:47)
	at breeze.generic.UFunc.apply$(UFunc.scala:46)
	at breez